## twohop in-context test

In [1]:
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from collections import deque
from transformers import AutoTokenizer
import pdb
import json
from scipy.stats import normaltest
import os
# print(os.getcwd())
from torch.nn import functional as F

import random
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from collections import deque
from transformers import AutoTokenizer
import pdb
from tqdm import tqdm
from torch.nn import functional as F
from src.llm_eval_configs import MODEL_OPTIONS, proto_template, short_names, mixed_locations, mixed_biology, languages

from scipy.stats import normaltest
%load_ext autoreload
%autoreload 2

"""values that you need to change"""
PROJECT_PATH = "." # the path of your working directory
access_token = None


/home/jovyan/project/twohopIC/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
def extract_ends(values):
    k = 1
    end_probs = []
    while 3*k - 1 < len(values):
        end_probs.append(values[3*k-1])
        k += 1
    summarize_probs = [end_probs[0].item(), torch.mean(torch.tensor(end_probs[1:])).item()]
    return summarize_probs

In [21]:
probs = {}
summaries = {}
for model, model_config in MODEL_OPTIONS.items():
    probs[model] = torch.load(os.path.join(model_config["dirname"], "tracked_prob.pt"))
    summaries[model] = {}
    for i in range(1, 6):
        summaries[model][i] = (extract_ends(probs[model][str(i)]['overall']['mean']), extract_ends(probs[model][str(i)]['overall']['std_error']))

In [22]:
summaries

{'qwen': {1: ([0.8017578125, nan], [0.00525665283203125, nan]),
  2: ([0.289794921875, 0.273193359375],
   [0.0092010498046875, 0.00751495361328125]),
  3: ([0.260498046875, 0.10418701171875],
   [0.00839996337890625, 0.003387451171875]),
  4: ([0.2578125, 0.064208984375],
   [0.00839996337890625, 0.002399444580078125]),
  5: ([0.239013671875, 0.04974365234375],
   [0.007778167724609375, 0.0020847320556640625])},
 'llama3-8b': {1: ([0.900390625, nan], [0.0022487640380859375, nan]),
  2: ([0.398193359375, 0.36962890625],
   [0.007781982421875, 0.0068206787109375]),
  3: ([0.318115234375, 0.19482421875],
   [0.0073089599609375, 0.004718780517578125]),
  4: ([0.28564453125, 0.1290283203125],
   [0.007328033447265625, 0.0033817291259765625]),
  5: ([0.268798828125, 0.1004638671875],
   [0.007297515869140625, 0.002719879150390625])},
 'llama3-70b': {1: ([0.87255859375, nan], [0.005290985107421875, nan]),
  2: ([0.4833984375, 0.171875], [0.0069427490234375, 0.0038661956787109375]),
  3: ([0.

In [24]:
probs = {}
summaries_lora = {}
for model, model_config in MODEL_OPTIONS.items():
    try:
        probs[model] = torch.load(os.path.join(model_config["dirname"], "tracked_prob_lora.pt"))
        summaries_lora[model] = {}
        for i in range(1, 6):
            summaries_lora[model][i] = (extract_ends(probs[model][str(i)]['overall']['mean']), extract_ends(probs[model][str(i)]['overall']['std_error']))
    except:
        continue

In [25]:
summaries_lora

{'qwen': {1: ([1.0, nan], [1.0728836059570312e-06, nan]),
  2: ([1.0, 0.0020008087158203125],
   [1.0728836059570312e-06, 0.0014133453369140625]),
  3: ([0.99658203125, 0.005157470703125],
   [0.0017547607421875, 0.0022411346435546875]),
  4: ([0.9951171875, 0.003566741943359375],
   [0.002056121826171875, 0.0018062591552734375]),
  5: ([0.990234375, 0.0041961669921875],
   [0.002902984619140625, 0.0019350051879882812])},
 'llama3-8b': {1: ([1.0, nan], [2.2649765014648438e-06, nan]),
  2: ([1.0, 0.0020008087158203125],
   [4.76837158203125e-07, 0.0014133453369140625]),
  3: ([0.99755859375, 0.0041046142578125],
   [0.001316070556640625, 0.00196075439453125]),
  4: ([0.9970703125, 0.003299713134765625],
   [0.0016508102416992188, 0.0017652511596679688]),
  5: ([0.99609375, 0.00469970703125],
   [0.0018815994262695312, 0.0020694732666015625])},
 'olmo': {1: ([1.0, nan], [2.2172927856445312e-05, nan]),
  2: ([1.0, 5.960464477539063e-08],
   [5.900859832763672e-06, 5.960464477539063e-08]),